# Import

In [ ]:
!pip install catboost pytorch-lightning pytorch-forecasting optuna -q

In [ ]:
# Drive Mount & Path Setup
import os
from google.colab import drive
drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/LGAI'
MODEL_DIR = os.path.join(BASE_DIR, 'models')
SUB_DIR = os.path.join(BASE_DIR, 'submissions')
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(SUB_DIR, exist_ok=True)

train_path  = os.path.join(BASE_DIR, 'train', 'train.csv')
sample_path = os.path.join(BASE_DIR, 'sample_submission.csv')
test_paths  = [os.path.join(BASE_DIR, 'test', f'TEST_0{i}.csv') for i in range(10)]

In [ ]:
# ---- Model Selection (Load or Train) ----from datetime import datetimedef list_models(ext):    files = sorted([f for f in os.listdir(MODEL_DIR) if f.endswith(ext)])    return filescat_files = list_models('.cbm')xgb_files = list_models('.json')tft_files = list_models('.ckpt')if cat_files:    print('Available CatBoost models:', cat_files)else:    print('No saved CatBoost model found.')cat_choice = input('Enter CatBoost filename to load or press Enter to train: ').strip()if cat_choice == '' or cat_choice == 'train' or cat_choice not in cat_files:    cat_choice = Noneif xgb_files:    print('Available XGBoost models:', xgb_files)else:    print('No saved XGBoost model found.')xgb_choice = input('Enter XGBoost filename to load or press Enter to train: ').strip()if xgb_choice == '' or xgb_choice == 'train' or xgb_choice not in xgb_files:    xgb_choice = Noneif tft_files:    print('Available TFT-mini models:', tft_files)else:    print('No saved TFT-mini model found.')tft_choice = input('Enter TFT-mini filename to load or press Enter to train: ').strip()if tft_choice == '' or tft_choice == 'train' or tft_choice not in tft_files:    tft_choice = None

In [ ]:

import os, gc, math, random
import numpy as np
import pandas as pd
from datetime import timedelta, datetime

import torch
import lightning as L
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger
import joblib
import json

from catboost import CatBoostRegressor, Pool

from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.metrics import RMSE
from pytorch_forecasting.data import GroupNormalizer

import optuna
import glob, re
from tqdm import tqdm
import logging

logging.getLogger("lightning").setLevel(logging.ERROR)
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)


# Fixed RandomSeed & Setting Hyperparameter

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

In [ ]:
# 공통 설정
ENC_LEN  = 28
PRED_LEN = 7
ROLL_WINS = [7, 14, 28]

# CatBoost 하이퍼파라미터
CAT_PARAMS = dict(
    depth=8,
    learning_rate=0.05,
    iterations=2000,
    loss_function="RMSE",
    l2_leaf_reg=5,
    random_seed=42,
    verbose=200,
    od_type="Iter",
    od_wait=200,
)

# TFT-mini 하이퍼파라미터 (dropout tuned later)
TFT_PARAMS = dict(
    learning_rate=2e-3,
    hidden_size=64,
    attention_head_size=2,
    hidden_continuous_size=32,
    lstm_layers=1,
)

# 앙상블 가중치
W_CAT = 0.4
W_TFT = 0.6


# Data Load

In [ ]:
train  = pd.read_csv(train_path, parse_dates=['영업일자'])
sample = pd.read_csv(sample_path)
tests  = {f'TEST_0{i}': pd.read_csv(p, parse_dates=['영업일자']) for i, p in enumerate(test_paths)}

# Define Model

# Train

In [ ]:
# --------------------------------------------------------------
# (1) 미래에 '알고 있는' 달력/이벤트 피처 목록
# --------------------------------------------------------------
known_future_cols = [
    "dow","month","is_weekend","dow_sin","dow_cos","month_sin","month_cos",
    "is_spring","is_summer","is_fall","is_winter",
    "is_peak_summer","is_peak_winter",
    "is_holiday","before_holiday","after_holiday","is_holiday_run",
    "is_summer_vac","is_winter_vac",
    "EVENT_SF_SZN","EVENT_SUMMER_SZN","EVENT_WINTER_SZN",
    "is_event_global","near_event_global","is_event_target","near_event_target",
]

# --------------------------------------------------------------
# (2) CatBoost 학습용: shift 벡터화 + fragmentation 방지
# --------------------------------------------------------------
train_sorted = train.sort_values(["영업장명_메뉴명","영업일자"]).reset_index(drop=True)
g = train_sorted.groupby("영업장명_메뉴명", sort=False)
train_sorted["hist_ok"] = g.cumcount() >= (ENC_LEN - 1)

base_for_shift = ["매출수량"] + known_future_cols
shift_blocks = []
for h in range(1, PRED_LEN + 1):
    block = g[base_for_shift].shift(-h)
    rename_map = {"매출수량": f"y_H{h}"}
    rename_map.update({c: f"{c}_H{h}" for c in known_future_cols})
    block = block.rename(columns=rename_map)
    shift_blocks.append(block)

train_shift = pd.concat(shift_blocks, axis=1)
train_sorted = pd.concat([train_sorted, train_shift], axis=1).copy()

def build_catboost_Xy_by_shift(df: pd.DataFrame, stride: int = 1):
    Xy = {}
    base_cols = cat_features_cols + [f"roll_mean_{w}" for w in ROLL_WINS]
    for c in base_cols:
        if c in cat_features_cols:
            df[c] = df[c].astype("category")
        else:
            df[c] = pd.to_numeric(df[c], errors="coerce").astype("float32")
    for h in range(1, PRED_LEN + 1):
        fut_cols_h = [f"{c}_H{h}" for c in known_future_cols]
        target_col = f"y_H{h}"
        mask = df["hist_ok"] & df[target_col].notna()
        use = df.loc[mask, base_cols + fut_cols_h + [target_col]].copy() if stride==1 else (
            df.loc[
                df[mask].groupby("영업장명_메뉴명", sort=False).apply(lambda x: x.iloc[::stride]).reset_index(level=0, drop=True).index,
                base_cols + fut_cols_h + [target_col]
            ].copy()
        )
        for c in fut_cols_h:
            use[c] = pd.to_numeric(use[c], errors="coerce").astype("float32")
        use[target_col] = pd.to_numeric(use[target_col], errors="coerce").astype("float32")
        X = use[base_cols + fut_cols_h].reset_index(drop=True)
        y = use[target_col].reset_index(drop=True)
        Xy[h] = (X, y)
    return Xy

Xy_h = build_catboost_Xy_by_shift(train_sorted, stride=1)

# --------------------------------------------------------------
# (3) TFT용 데이터셋/로더
# --------------------------------------------------------------
train_tft = train.copy()
train_tft["time_idx"] = (train_tft["영업일자"] - train_tft["영업일자"].min()).dt.days.astype(int)
for c in ["store_id", "item_id", "pair_id"]:
    train_tft[c] = train_tft[c].astype(str)

static_categoricals = ["store_id", "item_id", "pair_id"]

time_varying_known_reals = [
    "time_idx",
    "dow_sin","dow_cos","month_sin","month_cos",
    "is_weekend","is_spring","is_summer","is_fall","is_winter",
    "is_peak_summer","is_peak_winter",
    "is_holiday","before_holiday","after_holiday","is_holiday_run",
    "is_summer_vac","is_winter_vac",
    "EVENT_SF_SZN","EVENT_SUMMER_SZN","EVENT_WINTER_SZN",
    "is_event_global","near_event_global","is_event_target","near_event_target",
]

time_varying_unknown_reals = ["매출수량"] + [f"roll_mean_{w}" for w in ROLL_WINS]

for c in time_varying_known_reals:
    if c != "time_idx":
        train_tft[c] = pd.to_numeric(train_tft[c], errors="coerce").astype("float32")
train_tft["time_idx"] = pd.to_numeric(train_tft["time_idx"], errors="coerce").astype("int64")
for c in [f"roll_mean_{w}" for w in ROLL_WINS] + ["매출수량"]:
    train_tft[c] = pd.to_numeric(train_tft[c], errors="coerce").astype("float32")

training_cutoff = train_tft["time_idx"].max() - PRED_LEN

# --- Modified: added GroupNormalizer ---
tft_dataset = TimeSeriesDataSet(
    train_tft[train_tft.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="매출수량",
    group_ids=["pair_id"],
    min_encoder_length=ENC_LEN,
    max_encoder_length=ENC_LEN,
    min_prediction_length=PRED_LEN,
    max_prediction_length=PRED_LEN,
    static_categoricals=static_categoricals,
    time_varying_known_reals=time_varying_known_reals,
    time_varying_unknown_reals=time_varying_unknown_reals,
    target_normalizer=GroupNormalizer(groups=["pair_id"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

validation = TimeSeriesDataSet.from_dataset(tft_dataset, train_tft, predict=True, stop_randomization=True)
train_loader = tft_dataset.to_dataloader(train=True, batch_size=256, num_workers=2)
val_loader   = validation.to_dataloader(train=False, batch_size=256, num_workers=2)

print('[Define Dataset] dataset ready.')


In [ ]:
# ==============================================================# XGBoost 학습 (h=1..7) -- Added# ==============================================================import xgboost as xgbif xgb_choice:    xgb_models = joblib.load(os.path.join(MODEL_DIR, xgb_choice))    print(f"[XGBoost] Loaded {xgb_choice}")else:    xgb_models = {}    for h in range(1, PRED_LEN + 1):        Xh, yh = Xy_h[h]        if len(Xh) == 0:            print(f"[XGBoost][H{h}] 학습 데이터가 없습니다. 건너뜁니다.")            continue        X_tr, y_tr, X_va, y_va = split_train_valid(Xh, yh, valid_ratio=0.1)        print(f"[XGBoost][H{h}] train={len(X_tr):,}  valid={len(X_va):,}")        model = xgb.XGBRegressor(            max_depth=8,            learning_rate=0.05,            n_estimators=2000,            subsample=0.8,            colsample_bytree=0.8,            reg_lambda=5,            objective='reg:squarederror',            tree_method='hist',            random_state=42,            enable_categorical=True,        )        model.fit(X_tr, y_tr,                  eval_set=[(X_va, y_va)],                  verbose=200,                  early_stopping_rounds=100)        xgb_models[h] = model        gc.collect()    ts = datetime.now().strftime("%Y-%m-%d_%H-%M")    xgb_path = os.path.join(MODEL_DIR, f'xgb_model_{ts}.json')    joblib.dump(xgb_models, xgb_path)    print('[XGBoost] Saved model to', xgb_path)

In [ ]:
# ==============================================================# CatBoost 학습 (h=1..7)# ==============================================================if cat_choice:    cat_models = joblib.load(os.path.join(MODEL_DIR, cat_choice))    print(f"[CatBoost] Loaded {cat_choice}")else:    if "split_train_valid" not in globals():        def split_train_valid(X: pd.DataFrame, y: pd.Series, valid_ratio=0.1):            n = len(X); k = int(n * (1 - valid_ratio))            return (X.iloc[:k].reset_index(drop=True), y.iloc[:k].reset_index(drop=True),                    X.iloc[k:].reset_index(drop=True), y.iloc[k:].reset_index(drop=True))    if "cat_col_indices" not in globals():        def cat_col_indices(cols):            return [i for i, c in enumerate(cols) if c in cat_features_cols]    cat_models = {}    for h in range(1, PRED_LEN + 1):        Xh, yh = Xy_h[h]        if len(Xh) == 0:            print(f"[CatBoost][H{h}] 학습 데이터가 없습니다. 건너뜁니다.")            continue        cat_idx = cat_col_indices(Xh.columns)        X_tr, y_tr, X_va, y_va = split_train_valid(Xh, yh, valid_ratio=0.1)        print(f"[CatBoost][H{h}] train={len(X_tr):,}  valid={len(X_va):,}")        train_pool = Pool(X_tr, y_tr, cat_features=cat_idx)        valid_pool = Pool(X_va, y_va, cat_features=cat_idx)        cb = CatBoostRegressor(**CAT_PARAMS)        cb.fit(            train_pool,            eval_set=valid_pool,            use_best_model=True,            verbose=200        )        cat_models[h] = cb    gc.collect()    if torch.cuda.is_available():        torch.cuda.empty_cache()    ts = datetime.now().strftime("%Y-%m-%d_%H-%M")    cat_path = os.path.join(MODEL_DIR, f'catboost_model_{ts}.cbm')    joblib.dump(cat_models, cat_path)    print('[CatBoost] Saved model to', cat_path)

In [ ]:
# ==============================================================# TFT-mini 학습 (dropout 최적화 포함)# ==============================================================import optunaif tft_choice:    tft_model = TemporalFusionTransformer.load_from_checkpoint(os.path.join(MODEL_DIR, tft_choice))    print(f"[TFT] Loaded {tft_choice}")else:    # --- Modified: dropout optimization with Optuna ---    def tft_objective(trial):        dropout = trial.suggest_float('dropout', 0.1, 0.5)        model = TemporalFusionTransformer.from_dataset(            tft_dataset,            loss=RMSE(),            log_interval=200,            reduce_on_plateau_patience=4,            dropout=dropout,            weight_decay=1e-2,            **TFT_PARAMS        )        early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min')        trainer = L.Trainer(            max_epochs=30,            accelerator='gpu' if torch.cuda.is_available() else 'cpu',            devices=1,            precision='bf16-mixed' if torch.cuda.is_bf16_supported() else 32,            gradient_clip_val=0.1,            callbacks=[early_stop],            logger=False,            enable_progress_bar=False,            limit_val_batches=1.0,            num_sanity_val_steps=0,        )        trainer.fit(model, train_loader, val_loader)        return trainer.callback_metrics['val_loss'].item()    study = optuna.create_study(direction='minimize')    study.optimize(tft_objective, n_trials=10)    best_dropout = study.best_params['dropout']    print('Best dropout:', best_dropout)    # --- Train final model with optimal dropout ---    early_stop = EarlyStopping(monitor='val_loss', patience=6, mode='min')    lr_logger  = LearningRateMonitor(logging_interval='epoch')    checkpoint = ModelCheckpoint(monitor='val_loss', save_top_k=1, mode='min')    logger     = CSVLogger('tft_logs', name='catboost_tft')    precision = 'bf16-mixed' if torch.cuda.is_bf16_supported() else 32    print(f'[Precision] Using {precision}')    tft_model = TemporalFusionTransformer.from_dataset(        tft_dataset,        loss=RMSE(),        log_interval=200,        reduce_on_plateau_patience=4,        dropout=best_dropout,        weight_decay=1e-2,        **TFT_PARAMS    )    trainer = L.Trainer(        max_epochs=30,        accelerator='gpu' if torch.cuda.is_available() else 'cpu',        devices=1,        precision=precision,        gradient_clip_val=0.1,        callbacks=[early_stop, lr_logger, checkpoint],        logger=logger,        enable_progress_bar=True,        limit_val_batches=1.0,        num_sanity_val_steps=0,    )    trainer.fit(tft_model, train_dataloaders=train_loader, val_dataloaders=val_loader)    if checkpoint.best_model_path:        tft_model = TemporalFusionTransformer.load_from_checkpoint(checkpoint.best_model_path)        ts = datetime.now().strftime("%Y-%m-%d_%H-%M")        tft_path = os.path.join(MODEL_DIR, f'tft_model_{ts}.ckpt')        shutil.copy(checkpoint.best_model_path, tft_path)        print('[TFT] Saved model to', tft_path)    else:        print('[TFT] Warning: no best checkpoint found.')

# Prediction

In [ ]:
# ==============================================================
# Prediction Functions (CatBoost, TFT-mini, XGBoost) -- Modified
# ==============================================================

def convert_to_submission_format(pred_df: pd.DataFrame, sample_df: pd.DataFrame) -> pd.DataFrame:
    out = sample_df.copy()
    wide = pred_df.pivot(index='영업일자', columns='영업장명_메뉴명', values='매출수량')
    for r in range(len(out)):
        date_key = out.at[r, '영업일자']
        if date_key in wide.index:
            for c in out.columns:
                if c == '영업일자':
                    continue
                if c in wide.columns:
                    val = wide.at[date_key, c]
                    out.at[r, c] = 0 if pd.isna(val) else max(float(val), 0.0)
    for c in out.columns:
        if c == '영업일자':
            continue
        out[c] = pd.to_numeric(out[c], errors='coerce').fillna(0).clip(lower=0)
    return out


def prepare_test(df_test_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_test_raw.copy()
    df['영업장명_메뉴명'] = df['영업장명_메뉴명'].astype(str)
    df['업장명'] = df['영업장명_메뉴명'].str.split('_', n=1).str[0]
    df['메뉴명'] = df['영업장명_메뉴명'].str.split('_', n=1).str[1].fillna('NA')
    df['store_id'] = df['업장명'].map(store2id).fillna(-1).astype(int).astype(str)
    df['item_id']  = df['메뉴명'].map(item2id).fillna(-1).astype(int).astype(str)
    df['pair_id']  = (df['업장명'] + '###' + df['메뉴명']).map(pair2id).fillna(-1).astype(int).astype(str)
    df = df.sort_values(['영업장명_메뉴명','영업일자']).reset_index(drop=True)
    df = add_domain_features(df)
    for w in ROLL_WINS:
        df[f'roll_mean_{w}'] = (
            df.groupby('영업장명_메뉴명')['매출수량']
              .transform(lambda s: s.rolling(w, min_periods=1).mean())
        )
    return df

# --- CatBoost prediction ---
def predict_catboost(df_28: pd.DataFrame) -> pd.DataFrame:
    preds=[]
    for pair, sub in df_28.groupby('영업장명_메뉴명'):
        sub=sub.sort_values('영업일자').reset_index(drop=True)
        history=sub.iloc[-ENC_LEN:].copy()
        for h in range(1, PRED_LEN+1):
            fut_date = history['영업일자'].iloc[-1] + timedelta(days=1)
            fut_row=pd.DataFrame([{ '영업일자':fut_date,
                                    '영업장명_메뉴명':pair,
                                    '업장명':history['업장명'].iloc[-1],
                                    '메뉴명':history['메뉴명'].iloc[-1],
                                    'store_id':history['store_id'].iloc[-1],
                                    'item_id':history['item_id'].iloc[-1],
                                    'pair_id':history['pair_id'].iloc[-1],
                                    '매출수량':0 }])
            fut_row=add_domain_features(fut_row)
            tmp_hist=pd.concat([history, fut_row], ignore_index=True)
            for w in ROLL_WINS:
                tmp_hist[f'roll_mean_{w}'] = (
                    tmp_hist.groupby('영업장명_메뉴명')['매출수량']
                           .transform(lambda s: s.rolling(w, min_periods=1).mean())
                )
            fut_row=tmp_hist.iloc[[-1]]
            X=fut_row[cat_features_cols + [f'roll_mean_{w}' for w in ROLL_WINS]].copy()
            fut_feats=fut_row[known_future_cols].copy(); fut_feats.columns=[c+f'_H{h}' for c in fut_feats.columns]
            X=pd.concat([X.reset_index(drop=True), fut_feats.reset_index(drop=True)], axis=1)
            model=cat_models[h]; cat_idx=[i for i,c in enumerate(X.columns) if c in cat_features_cols]
            yhat=float(model.predict(Pool(X, cat_features=cat_idx))[0])
            yhat=max(0.0, yhat)
            preds.append({'영업장명_메뉴명':pair,'h':h,'pred_cat':yhat})
            fut_row.loc[:, '매출수량']=yhat
            history=pd.concat([history, fut_row], ignore_index=True)
    return pd.DataFrame(preds)

# --- XGBoost prediction ---
def predict_xgboost(df_28: pd.DataFrame) -> pd.DataFrame:
    preds=[]
    for pair, sub in df_28.groupby('영업장명_메뉴명'):
        sub=sub.sort_values('영업일자').reset_index(drop=True)
        history=sub.iloc[-ENC_LEN:].copy()
        for h in range(1, PRED_LEN+1):
            fut_date=history['영업일자'].iloc[-1] + timedelta(days=1)
            fut_row=pd.DataFrame([{ '영업일자':fut_date,
                                    '영업장명_메뉴명':pair,
                                    '업장명':history['업장명'].iloc[-1],
                                    '메뉴명':history['메뉴명'].iloc[-1],
                                    'store_id':history['store_id'].iloc[-1],
                                    'item_id':history['item_id'].iloc[-1],
                                    'pair_id':history['pair_id'].iloc[-1],
                                    '매출수량':0 }])
            fut_row=add_domain_features(fut_row)
            tmp_hist=pd.concat([history, fut_row], ignore_index=True)
            for w in ROLL_WINS:
                tmp_hist[f'roll_mean_{w}'] = (
                    tmp_hist.groupby('영업장명_메뉴명')['매출수량']
                           .transform(lambda s: s.rolling(w, min_periods=1).mean())
                )
            fut_row=tmp_hist.iloc[[-1]]
            X=fut_row[cat_features_cols + [f'roll_mean_{w}' for w in ROLL_WINS]].copy()
            fut_feats=fut_row[known_future_cols].copy(); fut_feats.columns=[c+f'_H{h}' for c in fut_feats.columns]
            X=pd.concat([X.reset_index(drop=True), fut_feats.reset_index(drop=True)], axis=1)
            model=xgb_models[h]
            yhat=float(model.predict(X)[0])
            yhat=max(0.0, yhat)
            preds.append({'영업장명_메뉴명':pair,'h':h,'pred_xgb':yhat})
            fut_row.loc[:, '매출수량']=yhat
            history=pd.concat([history, fut_row], ignore_index=True)
    return pd.DataFrame(preds)

# --- TFT prediction ---
def predict_tft(df_28: pd.DataFrame) -> pd.DataFrame:
    tmp=df_28.copy()
    tmp['time_idx']=(tmp['영업일자'] - train['영업일자'].min()).dt.days.astype(int)
    rows=[]
    for pair, sub in tmp.groupby('영업장명_메뉴명'):
        sub=sub.sort_values('영업일자')
        last_date=sub['영업일자'].iloc[-1]
        pid=sub['pair_id'].iloc[-1]; sid=sub['store_id'].iloc[-1]; iid=sub['item_id'].iloc[-1]
        for h in range(1, PRED_LEN+1):
            d=last_date + timedelta(days=h)
            rows.append({'영업장명_메뉴명':pair,'영업일자':d,'pair_id':pid,'store_id':sid,'item_id':iid})
    fut=pd.DataFrame(rows)
    fut=add_domain_features(fut.assign(매출수량=0))
    for w in ROLL_WINS:
        fut[f'roll_mean_{w}']=(
            fut.groupby('영업장명_메뉴명')['매출수량']
               .transform(lambda s: s.rolling(w, min_periods=1).mean())
               .fillna(0)
        )
    fut['time_idx']=(fut['영업일자'] - train['영업일자'].min()).dt.days.astype(int)
    enc_dec=pd.concat([tmp, fut], ignore_index=True)
    enc_dec=enc_dec.fillna(0)
    predict_ds=TimeSeriesDataSet.from_dataset(tft_dataset, enc_dec, predict=True, stop_randomization=True)
    predict_loader=predict_ds.to_dataloader(train=False, batch_size=256, num_workers=2)
    yhat=tft_model.predict(predict_loader, mode='prediction')
    if isinstance(yhat, tuple): yhat=yhat[0]
    fut_sorted=fut.sort_values(['pair_id','영업일자']).reset_index(drop=True)
    series_ids=fut_sorted['pair_id'].unique().tolist()
    out=[]
    for i,pid in enumerate(series_ids):
        pair_name=df_28.loc[df_28['pair_id']==pid, '영업장명_메뉴명'].iloc[0]
        for h in range(1, PRED_LEN+1):
            out.append({'영업장명_메뉴명':pair_name,'h':h,'pred_tft':float(max(0.0, yhat[i, h-1].item()))})
    return pd.DataFrame(out)

# --- helper to get predictions from all models ---
def predict_all_models(df_28: pd.DataFrame) -> pd.DataFrame:
    cat_pred=predict_catboost(df_28)
    tft_pred=predict_tft(df_28)
    xgb_pred=predict_xgboost(df_28)
    pred=cat_pred.merge(tft_pred, on=['영업장명_메뉴명','h'], how='outer')                 .merge(xgb_pred, on=['영업장명_메뉴명','h'], how='outer').fillna(0.0)
    return pred

# --- ensemble prediction using weights ---
def predict_ensemble_for_test_file(test_file_path: str, weights: dict) -> pd.DataFrame:
    df_raw=pd.read_csv(test_file_path, parse_dates=['영업일자'])
    df_28=prepare_test(df_raw)
    pred=predict_all_models(df_28)
    pred['pred_ens'] = (weights['w_cat']*pred.get('pred_cat',0) + weights['w_tft']*pred.get('pred_tft',0) + weights['w_xgb']*pred.get('pred_xgb',0))
    pred['file']=os.path.basename(test_file_path)
    return pred


In [ ]:
# ==============================================================
# Ensemble Weight Optimization (Optuna) -- Added
# ==============================================================

def build_validation_context(train_df):
    contexts=[]
    targets=[]
    for pair, sub in train_df.groupby('영업장명_메뉴명'):
        sub=sub.sort_values('영업일자')
        if len(sub) < ENC_LEN + PRED_LEN:
            continue
        hist=sub.iloc[-(ENC_LEN+PRED_LEN):-PRED_LEN].copy()
        fut=sub.iloc[-PRED_LEN:]['매출수량'].tolist()
        hist['영업장명_메뉴명']=pair
        contexts.append(hist)
        for h,val in enumerate(fut,1):
            targets.append({'영업장명_메뉴명':pair,'h':h,'y_true':float(val)})
    ctx=pd.concat(contexts).reset_index(drop=True)
    tgt=pd.DataFrame(targets)
    return ctx,tgt

val_ctx,val_tgt=build_validation_context(train)
val_pred=predict_all_models(val_ctx).merge(val_tgt, on=['영업장명_메뉴명','h'])
val_pred=val_pred.fillna(0.0)


def smape(y_true, y_pred):
    return np.mean(2*np.abs(y_true - y_pred)/(np.abs(y_true)+np.abs(y_pred)+1e-8))


def weight_objective(trial):
    w_cat=trial.suggest_float('w_cat',0,1)
    w_tft=trial.suggest_float('w_tft',0,1-w_cat)
    w_xgb=1 - w_cat - w_tft
    if w_xgb < 0:
        raise optuna.TrialPruned()
    y_hat=w_cat*val_pred['pred_cat'] + w_tft*val_pred['pred_tft'] + w_xgb*val_pred['pred_xgb']
    return smape(val_pred['y_true'], y_hat)

study_w=optuna.create_study(direction='minimize')
study_w.optimize(weight_objective, n_trials=50)
best_weights=study_w.best_params
best_weights['w_xgb']=1 - best_weights['w_cat'] - best_weights['w_tft']
print('Best weights:', best_weights)


In [ ]:
# -------------------------------
# Prediction 실행
# -------------------------------
all_preds = []
for test_path in sorted(test_paths):
    print(f'[Predict] {test_path}')
    all_preds.append(predict_ensemble_for_test_file(test_path, best_weights))
all_preds = pd.concat(all_preds, ignore_index=True)


# Submission

In [ ]:
from datetime import datetimesample_submission = pd.read_csv(sample_path)submission_df = convert_preds_to_submission(all_preds, sample_submission)ts = datetime.now().strftime("%Y-%m-%d_%H-%M")output_path = os.path.join(SUB_DIR, f'submission_{ts}.csv')submission_df.to_csv(output_path, index=False)weights_path = os.path.join(MODEL_DIR, f'ensemble_weights_{ts}.json')with open(weights_path, 'w') as f:    json.dump(best_weights, f)print('Saved submission to', output_path)print('Saved ensemble weights to', weights_path)